# Estimating Work from Home Model

This notebook illustrates how to re-estimate a single model component for ActivitySim.  This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [ ]:
import os
import larch  # !conda install larch -c conda-forge # for estimation
import pandas as pd
import numpy as np
from larch import P, X
import matplotlib.pyplot as plt

# Load data and prep model for estimation

In [ ]:
# change the below path to your output folder
edb_output_dir = r'C:\ABM3_dev\outputs'
os.chdir(edb_output_dir)
modelname = "transit_pass_ownership"

from activitysim.estimation.larch import component_model
model, data = component_model(modelname, return_data=True)

# Review data loaded from the EDB

The next step is to read the EDB, including the coefficients, model settings, utilities specification, and chooser and alternative data.

### Coefficients

In [ ]:
data.coefficients

#### Utility specification

In [ ]:
data.spec

### Chooser data

In [ ]:
data.chooser_data

# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [ ]:
model.load_data()

In [ ]:
model.maximize_loglike(method="SLSQP")

### Estimated coefficients

In [ ]:
model.parameter_summary()

# Output Estimation Results

In [ ]:
from activitysim.estimation.larch import update_coefficients
result_dir = data.edb_directory/"estimated"
update_coefficients(
    model, data, result_dir,
    output_file=f"{modelname}_coefficients_revised.csv",
)

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [ ]:
# can change result_dir to desired location
model.calculate_parameter_covariance()
model.to_xlsx(
    result_dir+"transit_pass_ownership.xlsx", 
    data_statistics=True,
)

# Next Steps

The final step is to either manually or automatically copy the `*_coefficients_revised.csv` file to the configs folder, rename it to `*_coefficients.csv`, and run ActivitySim in simulation mode.

In [ ]:
pd.read_csv(result_dir/f"{modelname}_coefficients_revised.csv")